In [1]:
import hashlib
import os
import csv
from datetime import datetime

def sha256_file(path):
    """Return SHA-256 hash of a file."""
    h = hashlib.sha256()
    with open(path, "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            h.update(chunk)
    return h.hexdigest()

def create_baseline(folder, output_csv="baseline.csv"):
    records = []
    for root, dirs, files in os.walk(folder):
        for f in files:
            full_path = os.path.join(root, f)
            file_hash = sha256_file(full_path)
            timestamp = datetime.now().isoformat()
            records.append([full_path, file_hash, timestamp])

    with open(output_csv, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["file", "hash", "timestamp"])
        writer.writerows(records)

    print(f"[+] Baseline created with {len(records)} files.")

# Example usage:
# create_baseline("test_folder")


In [2]:
import csv
import os
import hashlib

def sha256_file(path):
    h = hashlib.sha256()
    with open(path, "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            h.update(chunk)
    return h.hexdigest()

def load_baseline(baseline_csv):
    baseline = {}
    with open(baseline_csv) as f:
        reader = csv.DictReader(f)
        for row in reader:
            baseline[row["file"]] = row["hash"]
    return baseline

def detect_changes(folder, baseline_csv="baseline.csv"):
    baseline = load_baseline(baseline_csv)

    modified = []
    deleted = []
    new_files = []

    # Check current files
    for root, dirs, files in os.walk(folder):
        for f in files:
            full_path = os.path.join(root, f)
            current_hash = sha256_file(full_path)

            if full_path not in baseline:
                new_files.append(full_path)
            elif baseline[full_path] != current_hash:
                modified.append(full_path)

    # Check for deleted files
    for file in baseline:
        if not os.path.exists(file):
            deleted.append(file)

    print("\n=== Suspicious File Changes ===")
    print("Modified:", modified)
    print("New:", new_files)
    print("Deleted:", deleted)

# Example usage:
# detect_changes("test_folder")


In [3]:
import os
import re

SIGNATURES = [
    r"eval\(",
    r"exec\(",
    r"base64\.b64decode",
    r"socket\.connect",
    r"import os",
]

def scan_file(path):
    findings = []
    try:
        with open(path, "r", errors="ignore") as f:
            content = f.read()
            for sig in SIGNATURES:
                if re.search(sig, content):
                    findings.append(sig)
    except Exception:
        pass
    return findings

def scan_folder(folder):
    results = {}
    for root, dirs, files in os.walk(folder):
        for f in files:
            full_path = os.path.join(root, f)
            signatures = scan_file(full_path)
            if signatures:
                results[full_path] = signatures

    print("\n=== Signature Scan Results ===")
    for file, sigs in results.items():
        print(f"[!] Suspicious patterns found in {file}: {sigs}")

# Example usage:
# scan_folder("test_folder")


In [4]:
import random

def simulate_worm(num_hosts=20, infection_chance=0.3, steps=10):
    # 0 = clean, 1 = infected
    network = [0] * num_hosts
    network[0] = 1  # initial infection

    for step in range(steps):
        print(f"\n--- Step {step+1} ---")
        new_infections = []

        for host in range(num_hosts):
            if network[host] == 1:
                target = random.randint(0, num_hosts - 1)
                if random.random() < infection_chance:
                    new_infections.append(target)

        for i in new_infections:
            network[i] = 1

        print("Infected hosts:", [i for i, state in enumerate(network) if state == 1])

    return network

# Example usage:
# simulate_worm()
